In [12]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
model_path = '../models/pose_landmarker_lite.task'
video_path = '../data/20230514192027.mp4'

In [13]:
import cv2
movie = cv2.VideoCapture(video_path)
movie_fps = movie.get(cv2.CAP_PROP_FPS)
movie_frames = movie.get(cv2.CAP_PROP_FRAME_COUNT)
initial_timestamp = 0

In [15]:
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a pose landmarker instance with the video mode:
options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.VIDEO)

with PoseLandmarker.create_from_options(options) as landmarker:
    for f in range(int(movie_frames)):
        frame_timestamp_ms = initial_timestamp + (f * 1000 / movie_fps)
        success, image = movie.read()
        if not success:
            break
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
        pose_landmarker_result = landmarker.detect_for_video(mp_image, frame_timestamp_ms)

TypeError: at(): incompatible function arguments. The following argument types are supported:
    1. (self: mediapipe.python._framework_bindings.packet.Packet, arg0: int) -> mediapipe.python._framework_bindings.packet.Packet
    2. (self: mediapipe.python._framework_bindings.packet.Packet, arg0: mediapipe.python._framework_bindings.timestamp.Timestamp) -> mediapipe.python._framework_bindings.packet.Packet

Invoked with: <mediapipe.Packet with timestamp: UNSET and C++ type: ::mediapipe::Image>, 0.0